In [ ]:
'''
original code source : https://boostcampaitech.slack.com/archives/C049SV5AXK2/p1670807640700879 (Thanks to T4047 김종해)
'''

In [ ]:
import json
import pandas as pd

In [ ]:
icd_path = '/opt/ml/input/data/ICDAR17_Korean/ufo/train.json'
drv_path = '/opt/ml/input/data/Upstage/ufo/train.json'
save_path = '/opt/ml/input/data/Upstage/ufo/new_train.json'

with open(icd_path, 'r') as f:
    icd = json.load(f)
with open(drv_path, 'r') as f:
    drv = json.load(f)

### 최상위 레벨의 keys 비교

In [ ]:
print(icd.keys())
print(drv.keys())

In [ ]:
print(drv['version'])
print(drv['tags'])

In [ ]:
ext_icd = set()
ext_drv = set()

for i in icd['images'].keys():
    ext_icd.add(i.split('.')[-1])
for i in drv['images'].keys():
    ext_drv.add(i.split('.')[-1])
    
print(ext_icd)
print(ext_drv)

### 이미지 1장 레벨의 keys 비교

In [ ]:
print(icd['images']['img_4380.jpg'].keys())
print(drv['images']['0F885DC0-3E65-4081-9DBB-CA96BB6FD4FC.JPG'].keys())

In [ ]:
allimg_drv = pd.DataFrame(drv['images']).T

print(sum(allimg_drv['paragraphs'] != {}))
print(sum(allimg_drv['chars'] != {}))
print(sum(allimg_drv['relations'] != {}))
print(allimg_drv['annotation_log'].apply(lambda x : tuple(x.items())).value_counts())


In [ ]:
# img_h, img_w, words, tags, license_tag 살펴보기 (공통레벨)

com_icd = pd.DataFrame(icd['images']).T[['img_h', 'img_w', 'words', 'tags', 'license_tag']]
com_drv = pd.DataFrame(drv['images']).T[['img_h', 'img_w', 'words', 'tags', 'license_tag']]

In [ ]:
## 1. img_h, img_w
print(com_icd['img_h'].unique())
print(com_icd['img_w'].unique())
print(com_drv['img_h'].unique())
print(com_drv['img_w'].unique())

## 특이사항 없음

In [ ]:
## 2. words
ann_icd = pd.concat({k: pd.DataFrame(v) for k, v in com_icd['words'].items()}, axis=1).T
ann_drv = pd.concat({k: pd.DataFrame(v) for k, v in com_drv['words'].items()}, axis=1).T

print(ann_icd.columns)
print(ann_drv.columns)

In [ ]:
## 2-1. words _ word_tags, tags, confidence (차이)

print(sum(ann_icd['word_tags'].notnull()))
## ann_icd 중 word_tags는 모두 null (삭제 or 무시)

print(sum(ann_drv['confidence'].notnull()))

exc_reg_list = ann_drv['tags'][ann_drv['tags'].apply(lambda x : len(x)) != 0][ann_drv['tags'][ann_drv['tags'].apply(lambda x : len(x)) != 0].apply(lambda x : x[0]) == 'excluded-region'].index
no_trans_list = ann_drv['transcription'][ann_drv['transcription'].isnull()].index
for i in range(len(exc_reg_list)):
    if exc_reg_list[i] != no_trans_list[i]:
        print(i)
print(len(exc_reg_list), len(no_trans_list))
## ann_drv중 confidence는 모두 null (삭제 or 무시)
## ann_drv 중 'tags'에 'excluded-region'는 'transcription'이 None인 것과 완벽히 일치
## transcription이 None인 것을 지우면 'tags'-'excluded-region'은 신경쓰지 않아도 될듯

In [ ]:
## 2-2. words _ points, transcription, language, illegibility, orientation (공통)


print('-'*30, '<words _ points>', sep='\n', end='\n\n')
## 2-2-1. words _ points (icd_4078, drv_23627)
print(sum(ann_icd['points'].apply(lambda x : type(x)) != type([])))
print(ann_icd['points'].apply(lambda x : len(x)).value_counts())
## points는 모두 list 타입
## 빈 값은 없으며, 모두 4개의 점으로 구성

print("-------------------------------------------------------------")
print(sum(ann_drv['points'].apply(lambda x : type(x)) != type([])))
print(sum(ann_drv['points'].apply(lambda x : len(x)).value_counts()))
print(ann_drv['points'].apply(lambda x : len(x)).value_counts())
## points는 모두 list 타입
## 빈 값은 없으며, 점의 개수는 다양 (모두 짝수)

In [ ]:
print('-'*30, '<words _ transcription, illegibility>', sep='\n', end='\n\n')
## 2-2-2. words _ transcription, illegibility
print(ann_icd['illegibility'].unique())
none_ann_list = ann_icd[ann_icd['transcription'] == '###'].index
ille_ann_list = ann_icd[ann_icd['illegibility'] == True].index
for i in range(len(none_ann_list)):
    if none_ann_list[i] != ille_ann_list[i]:
        print(i)
print(len(none_ann_list), len(ille_ann_list))
## illegibility는 True, False만 보유
## transcription이 ###인 데이터는 illegibility가 True (transcription을 위주로 살펴도 될듯)
print(ann_drv['illegibility'].unique())
print(ann_drv[ann_drv['illegibility'] == True]['transcription'].unique())
## illegibility는 True, False만 보유
## illegibility가 True인 것은 transcription이 None인 것을 포함하지만, None이 아닌 것들은 거의 읽기 불가능한 수준 (무시가능)

In [ ]:
print('-'*30, '<words _ language>', sep='\n', end='\n\n')
## 2-2-3. words _ language
print(sum(ann_icd['language'].apply(lambda x : len(x)) != 1))
print(ann_icd['language'].apply(lambda x : x[0]).value_counts())
## language는 ko 또는 en이며, 2개 이상의 값은 없음
print(sum(ann_drv['language'].isnull()))
print(ann_drv[ann_drv['language'].notnull()]['language'].apply(lambda x : tuple(x)).value_counts())
## language는 null 또는 ko, en, others의 조합 (2개 이상 값 존재)


print('\n', '-'*30, '<words _ orientation>', sep='\n', end='\n\n')
## 2-2-4. words _ orientation
print(ann_icd['orientation'].value_counts())
## 모든 글자방향이 Horizontal
print(ann_drv['orientation'].value_counts())
## 글자방향은 Horizontal, Vertical, Irregular이 존재

In [ ]:
## 3. tags, license_tag

print(sum(com_icd['tags'].notnull()))
print(com_drv['tags'].apply(lambda x : tuple(x)).value_counts())
print(sum(com_drv['tags'].apply(lambda x : len(x)) != 0) / len(com_drv))
## com_icd는 null만을 소유
## com_drv의 8%가 tags를 소유, 학습하는건 크게 의미가 없을듯
## 오류분석 시 활용될 수도 있을듯

print('\n', '-'*30)
print(com_icd['license_tag'].apply(lambda x : tuple(x.items())).value_counts())
print(com_drv['license_tag'].apply(lambda x : tuple(x.items())).value_counts())
## license_tag는 별다른 내용을 갖지 않음 (무시 가능할듯)

In [ ]:
# 최소구조

# images
### filename
##### img_h
##### img_w
##### words
####### points
####### transcription
####### language
####### illegibility
####### orientation
##### tags
##### license_tag

### words가 없는 images 제거

In [ ]:
com_drv['words'][com_drv['words'].apply(lambda x : x.values()).apply(lambda x : len(x)) == 0]

### point가 4개 이상인 polygon 확인

In [ ]:
for i in drv['images'].keys():
    for key in drv['images'][i]['words'].keys():
        if len(drv['images'][i]['words'][key]['points']) != 4:
            print(len(drv['images'][i]['words'][key]['points']))

In [ ]:
# words가 없는 이미지를 삭제한 dict 생성
import copy
temp_drv = copy.deepcopy(drv)

for i in drv['images'].keys():
    for key in drv['images'][i]['words'].keys():
        if len(drv['images'][i]['words'][key]['points']) != 4:
            # print(len(drv['images'][i]['words'][key]['points']))
            del temp_drv['images'][i]['words'][key]        

result_drv = copy.deepcopy(temp_drv)

for i in temp_drv['images'].keys():
    if len(temp_drv['images'][i]['words'].keys()) == 0:
        del result_drv['images'][i]


# for i in com_drv['words'][com_drv['words'].apply(lambda x : x.values()).apply(lambda x : len(x)) == 0].keys():
#     del new_drv['images'][i]
    
test = pd.DataFrame(result_drv['images']).T[['img_h', 'img_w', 'words', 'tags', 'license_tag']]
test

### 포인트 4개인 박스만 남았는지 확인

In [ ]:
# 출력되는 값이 없어야 함
for i in result_drv['images'].keys():
    for key in result_drv['images'][i]['words'].keys():
        if len(result_drv['images'][i]['words'][key]['points']) != 4:
            print(len(result_drv['images'][i]['words'][key]['points']))

### word가 없는 이미지 제거 확인

In [ ]:
# 출력되는 값이 없어야 함
for i in result_drv['images'].keys():
    if len(result_drv['images'][i]['words'].keys()) == 0:
        print(i)

In [ ]:
# 현재 경로에 'new_train.json'으로 저장
with open(save_path, 'w') as f:
    json.dump(result_drv, f, indent=2)